# 🚀 Spaceship Titanic 발표 대본

이 노트북은 `Spaceship_3_수정전.ipynb` 파일을 기반으로 발표를 진행하기 위한 대본입니다.
각 셀은 발표의 주요 흐름을 담당하며, 원본 노트북의 해당 섹션을 띄워놓고 설명할 내용을 담고 있습니다.

## 1. 오프닝 (Introduction)

**[화면: 노트북의 '소개' 및 '데이터셋 가져오기' 부분]**

안녕하십니까, Spaceship Titanic 생존자 예측 프로젝트 발표를 시작하겠습니다.
이번 프로젝트의 목표는 4광년 떨어진 곳으로 향하다가 시공간 이상 현상과 충돌해 다른 차원으로 전송된 승객들을 예측하는 것입니다.

저희는 약 13,000명의 승객 데이터를 분석하여, 어떤 승객이 전송되었는지 머신러닝 모델을 통해 예측해보았습니다.

## 2. 데이터 탐색 (EDA)

**[화면: '탐색적 데이터 분석(EDA)' 섹션, 특히 Target 분포 그래프]**

먼저 데이터를 살펴보았습니다.
타겟 변수인 `Transported`(전송 여부)를 확인해본 결과, True와 False가 거의 5:5 비율로 균형을 이루고 있어 데이터 불균형 문제는 없었습니다.

**[화면: '동면 여부(CryoSleep)' 시각화 부분]**

각 특성별로 생존율(전송될 확률)을 분석했습니다.
가장 눈에 띄는 것은 `CryoSleep`(동면) 여부였습니다. 동면 중인 승객은 전송될 확률이 80% 이상으로 매우 높았고, 깨어있는 승객은 30% 정도로 낮았습니다. 이는 동면 상태가 생존 여부에 큰 영향을 미친다는 것을 시사합니다.

**[화면: '나이(Age)' 및 'AgeGroup' 시각화 부분]**

나이대별 분석에서는 0~4세 영유아의 전송 확률이 매우 높았고, 반대로 나이가 많은 층은 확률이 낮아지는 경향을 보였습니다.

## 3. 특성 공학 (Feature Engineering)

**[화면: '특성 엔지니어링' 섹션 - FamilySize 및 파생 변수 생성 코드]**

모델의 성능을 높이기 위해 몇 가지 파생 변수를 생성했습니다.

1.  **FamilySize**: 승객의 성씨(Surname)를 추출하여 가족 규모를 파악했습니다. 가족이 함께 움직였을 가능성을 고려했습니다. (다만, 단순히 성씨만으로 묶는 것은 동명이인의 오류가 있을 수 있어 추후 그룹ID와 결합하는 방식으로 개선할 여지가 있습니다.)
2.  **IsSleepZero**: 동면 중인데 지출이 있거나 하는 데이터의 모순을 해결하기 위해, 지출이 0인 경우를 명확히 하는 특성을 고려했습니다.
3.  **IsAlone**: 혼자 탑승한 승객인지 여부도 중요한 변수로 사용했습니다.
4.  **IsVIPDeck**: 특정 데크(B, C, T) 등 부유층이 머무는 구역에 대한 정보를 변수화했습니다.

또한 나이를 `Baby`, `Teenager`, `Adult` 등으로 범주화하여 `AgeGroup` 변수를 만들었습니다.

## 4. 데이터 전처리 (Preprocessing)

**[화면: '데이터 전처리' 섹션 - 결측치 처리 부분]**

결측치 처리에 많은 공을 들였습니다. 단순히 평균으로 채우기보다는 데이터의 맥락을 활용했습니다.

1.  **지출 금액(Spending)**: 결측치는 대부분 지출이 없는 경우라 판단하여 0으로 채웠습니다.
2.  **동면 여부(CryoSleep)**: 지출이 있다면 '깨어있음(False)', 지출이 0이라면 '동면(True)'으로 유추하여 채웠습니다.
3.  **목적지 & 고향**: 최빈값으로 채우거나, 같은 그룹(가족)의 정보를 가져와서 채우는 방식을 사용했습니다.
4.  **Cabin**: 가족이나 같은 그룹은 인접한 객실을 쓸 확률이 높으므로, 같은 그룹의 Deck 정보를 공유받도록 처리했습니다.

## 5. 모델링 및 결과 (Modeling)

**[화면: '머신러닝 모델 구축 및 평가' 섹션 - LGBM, XGBoost 코드]**

최종 예측을 위해 **LightGBM**과 **XGBoost** 모델을 사용했습니다.
단일 모델만 사용하는 것보다 성능이 우수한 부스팅 계열 모델을 선택했으며, **Optuna**를 사용하여 하이퍼파라미터 최적화를 진행했습니다.

**[화면: 모델 정확도 출력 부분]**

검증 결과, XGBoost 모델에서 약 0.81점대의 정확도를 확보할 수 있었습니다.
Optuna를 통해 최적의 학습률(learning_rate)과 트리 깊이(max_depth) 등을 찾아냈으며, 이를 통해 기본적인 모델보다 성능을 향상시켰습니다.

## 6. 결론 (Conclusion)

이번 프로젝트를 통해 결측치를 논리적으로 채우는 전처리 과정이 모델 성능에 큰 영향을 미친다는 것을 확인했습니다.
특히 '동면 여부'와 '지출 금액' 간의 관계를 이용한 결측치 처리가 유효했습니다.

이상으로 Spaceship Titanic 생존자 예측 발표를 마치겠습니다. 감사합니다.